In [1]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('clean_data.csv')
df

,Unnamed: 0,Street,Ward,District,Prices,Area,toFace,type,certificate,width,length,floors,Location
0,0,Hà Mục,Hòa Thọ Đông,cẩm lệ,5.15,100.0,Tây Nam,Nhà mặt tiền,Không có,5.0,20.0,4.0,"16.01918,108.19537"
1,1,Thi Sách,Hòa Thuận Tây,hải châu,4.10,85.0,Không,Nhà mặt tiền,Không có,NaN,NaN,1.0,"16.0546,108.20547"
2,2,Ông Ích Khiêm,Tam Thuận,thanh khê,11.00,67.0,Tây,Nhà mặt tiền,Giấy tờ hợp lệ,3.8,NaN,3.0,"16.06653,108.2142"
3,3,Núi Thành,Hòa Cường Bắc,hải châu,17.50,137.0,Tây,Nhà mặt tiền,Giấy tờ hợp lệ,7.3,NaN,3.0,"16.03194,108.22279"
4,4,Trương Văn Hiến,Khuê Mỹ,ngũ hành sơn,7.00,90.0,Đông Nam,Nhà mặt tiền,Giấy tờ hợp lệ,5.0,18.0,4.0,"16.01947,108.25592"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14607,14936,Nguyễn Thị Định,An Hải Bắc,sơn trà,9.50,90.0,Không,Nhà mặt tiền,Không có,NaN,NaN,3.0,"16.08277,108.23274"
14608,14937,300/22 Hải Phòng,Hải Châu I,hải châu,2.85,50.0,Tây Nam,Nhà trong hẻm,Giấy tờ hợp lệ,5.0,10.0,2.0,"16.0726,108.21948"
14609,14938,Ông Ích Khiêm,Thanh Bình,hải châu,4.35,57.0,Đông Bắc,Nhà trong hẻm,Không có,5.0,12.0,3.0,"16.07838,108.21186"
14610,14939,K814A/52 Trần Cao Vân,Thanh Khê Đông,thanh khê,3.35,5.0,Tây,Nhà trong hẻm,Giấy tờ hợp lệ,4.5,12.5,2.0,"16.07346,108.21122"


In [5]:
df['Location'].unique()

array(['16.01918,108.19537', '16.0546,108.20547', '16.06653,108.2142',
       ..., '16.08137,108.16071', '16.07346,108.21122',
       '16.0726,108.21948'], dtype=object)

In [ ]:
from geopy.geocoders import Nominatim


def get_Location(Address):

    geolocator = Nominatim(user_agent="my-app")

    # Lấy tọa độ từ địa ch
    location = geolocator.geocode(Address,timeout=10)
    # In ra tọa độ
    if location is not None:
        latitude = location.latitude
        longitude = location.longitude
    else:
        latitude = None
        longitude = None
    return (str(latitude)+","+ str(longitude))


In [ ]:
df['address'] = df['Street'] + ', ' + df['District']+', Đà Nẵng, Việt Nam'


In [ ]:
from sklearn.model_selection import train_test_split
house = df[['Prices','District','Area','type','certificate','width','length','floors']]
df_train, df_test = train_test_split(house, test_size = 0.2, random_state=5)

print(df_train.shape)
print(df_test.shape)
X_train = df_train.copy()
X_test = df_test.copy()

print("X_train: {0}".format(X_train.shape))
print("X_test: {0}".format(X_test.shape))
y_train = X_train.pop('Prices')
y_test = X_test.pop('Prices')
print("Y_train: {0}".format(y_train.shape))
print("y_test: {0}".format(y_test.shape))
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Dùng LinerRegression
model_LR = LinearRegression()
model_LR.fit(X_train, y_train)
predictions_LR  = model_LR.predict(X_test)
predictions_LR
from sklearn.metrics import r2_score, accuracy_score
r2_score_LR = r2_score(y_test,predictions_LR)
print(r2_score_LR)
rs = {
  "Price_Test": y_test[:10],
  "Price_Predict": predictions_LR[:10]
}

df_t = pd.DataFrame(rs)
print(df_t)


In [9]:
from bs4 import BeautifulSoup
import requests
from csv import writer

def get_Links_Detail_in_Page(HTML_page):
    linksDetail = []
    for k in HTML_page.find_all(class_='ct_title'):
        for j in k.find_all('a'):
            linksDetail.append(j.get('href'))
    return linksDetail

url = "https://www.danang43.vn/tong-hop-cac-con-duong-noi-tieng-o-da-nang-p632.html"
soup= BeautifulSoup(requests.get(url).text,'html.parser')

streets =  soup.find_all('td')

rs = [x.text for x in streets[1::2] ]


In [17]:
rs = [str(x).replace('Đường\xa0','') for x in rs]
arr = []
for x in rs:
    temp = "\'"+x+"\'"
    arr.append(temp)

with open('raw_data2.csv', 'a',encoding= "utf8",newline='') as f_object:
    writer_object = writer(f_object)
    writer_object.writerow(arr)

arr

["'Đường Cao Thắng'",
 "'Lê Đình Lý'",
 "'Hùng Vương'",
 "'Lê Lợi'",
 "'3.2'",
 "'Nguyễn Chí Thanh'",
 "'Phan Châu Trinh'",
 "'Tiểu La'",
 "'2.9'",
 "'Quang Trung'",
 "'Đống Đa'",
 "'Nguyễn Du'",
 "'Lý Tự Trọng'",
 "'Hoàng Diệu'",
 "'Ông Ích Khiêm'",
 "'Nguyễn Hoàng'",
 "'Thái Phiên'",
 "'Lê Hồng Phong'",
 "'Hoàng Văn Thụ'",
 "'Yên Bái'",
 "'Trưng Nữ Vương'",
 "'Ngô Gia Tự'",
 "'Triệu Nữ Vương'",
 "'Cao Thắng'",
 "'Bắc Đẩu'",
 "'Hải Hồ'",
 "'Pasteur'",
 "'Lê Đình Dương'",
 "'Nguyễn Văn Linh'",
 "'Lê Thanh Nghị'",
 "'Xô Viết Nghệ Tĩnh'",
 "'Trần Phú'",
 "'Bạch Đằng'",
 "'Nguyễn Tất Thành'",
 "'Thái Thị Bôi'",
 "'Lê Độ'",
 "'Trường Chinh'",
 "'Hải Phòng'",
 "'Điện Biên Phủ'",
 "'Hà Huy Tập'",
 "'Trần Cao Vân'",
 "'Nguyễn Tri Phương'",
 "'Lê Lợi'",
 "'Hùng Vương'",
 "'Lê Đình Lý'",
 "'Duy Tân'",
 "'Tiểu La'",
 "'Nguyễn Phước Nguyên'",
 "'Xô Viết Nghệ Tĩnh'",
 "'2.9'",
 "'Nguyễn Lương Bằng'",
 "'Nguyễn tất Thành'",
 "'Bàu Tràm'",
 "'Âu Cơ'",
 "'Đoàn Phú Thứ'",
 "'Nam Cao'",
 "'Lạc Long Quâ

In [2]:
import requests

# Địa chỉ cần tìm tọa độ
address = "Đường Huyền Trân Công Chúa, Ngũ Hành Sơn, Đà Nẵng, Việt Nam"

# Tạo URL request
url = "https://dev.virtualearth.net/REST/v1/Locations"

# Gửi yêu cầu HTTP
def getLocation(Address):
    params = {
        "q": Address,
        "key": "AuCD3WhuOOT5HhbRcvJAt1DyK293gMNSAfPSTcgaXGlrVlVTRpXs06BWdTexjaOA"
    }
    response = requests.get(url, params=params)
    data = response.json()
    # Kiểm tra xem có kết quả tọa độ hay không
    if "resourceSets" in data and len(data["resourceSets"]) > 0 and "resources" in data["resourceSets"][0] and len(data["resourceSets"][0]["resources"]) > 0:
        coordinates = data["resourceSets"][0]["resources"][0]["point"]["coordinates"]
        latitude = coordinates[0]
        longitude = coordinates[1]
        location = str(latitude)+","+str(longitude)
        return location
    else:
        return np.nan


Latitude: 16.00209
Longitude: 108.26342


In [ ]:
# from sklearn.model_selection import GridSearchCV

# grid_search = GridSearchCV(estimator=model_RD, param_grid=param_grid,cv=3, n_jobs=-1, verbose=2)
# # Fit the grid search to the data
# grid_fit = grid_search.fit(X_train, y_train)
# best_parameters = grid_fit.best_params_
# best_parameters